In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from aiida import load_profile

# ! Hackish way to be able to import from config_class in the parent directory
import sys
sys.path.append()

In [2]:
load_profile()

Profile<uuid='4fe00b7a32994cfca8b6cf43c2d22a53' name='jgeiger'>

***
# Sampling of configurations using `icet`

In [8]:
import os
import pandas as pd
from pybat import Cathode

from config_class import ConfigClass

ImportError: cannot import name 'ConfigClass' from 'config_class' (/home/jgeiger/projects/bat_uv_ml/code/notebooks/../config_class.py)

In [ ]:

fully_lithiated_df = pd.read_pickle(
    os.path.join('..', '..', "data", "fully_lithiated_df.pkl")
)

# Setup of configuration sampling
GLOBAL_SYMPREC = 1e-5
devel_ase = fully_lithiated_df["ase_in"].values[0]
devel_pmg = fully_lithiated_df["pmg_in"].values[0]
devel_cathode = Cathode.from_structure(devel_pmg)

# Set initial, general parameters
config_class = ConfigClass()
config_class.short_name = "LFPO"
config_class.add_structure(devel_cathode)
config_class.eval_parent_sg(symprec=GLOBAL_SYMPREC)
config_class.cell_size = 2

# Run configuration sampling
devel_config_dict = dict(
    substitution_sites=list(range(0, devel_ase.get_chemical_symbols().count("Li"))),
    cation_list=["Li", "Vac"],
    concentration_restrictions={"Li": (0, 1)},
    max_configurations=None,
    symprec=GLOBAL_SYMPREC,
)

# Create df which is linked to the class instance
config_class.create_configurations(**devel_config_dict)
config_class.create_dataframe()
print(config_class.data_df)